# Exercise sheet \#4
## Using APIs
### Exercise 1
Write a Python script which gets the list of astronauts who are currently in Space. To do so, you can use the [astro.json](http://api.open-notify.org/astros.json) OpenNotify API. 

In [1]:
import requests
import pprint

pp = pprint.PrettyPrinter(indent=4)

#Récupérer les données du site
response = requests.get("http://api.open-notify.org/astros.json")
print(response)
print("")
# Décoder format json pr le lire
data = response.json()
pp.pprint(data)
print("")
astronauts = [x['name'] for x in data['people']]
print(astronauts)

<Response [200]>

{   'message': 'success',
    'number': 10,
    'people': [   {'craft': 'ISS', 'name': 'Sergey Prokopyev'},
                  {'craft': 'ISS', 'name': 'Dmitry Petelin'},
                  {'craft': 'ISS', 'name': 'Frank Rubio'},
                  {'craft': 'ISS', 'name': 'Nicole Mann'},
                  {'craft': 'ISS', 'name': 'Josh Cassada'},
                  {'craft': 'ISS', 'name': 'Koichi Wakata'},
                  {'craft': 'ISS', 'name': 'Anna Kikina'},
                  {'craft': 'Shenzhou 15', 'name': 'Fei Junlong'},
                  {'craft': 'Shenzhou 15', 'name': 'Deng Qingming'},
                  {'craft': 'Shenzhou 15', 'name': 'Zhang Lu'}]}

['Sergey Prokopyev', 'Dmitry Petelin', 'Frank Rubio', 'Nicole Mann', 'Josh Cassada', 'Koichi Wakata', 'Anna Kikina', 'Fei Junlong', 'Deng Qingming', 'Zhang Lu']


### Exercise 2

#### Question 2.1
Write a Python program, which, for each astronaut found in Exercise 1, retrieves  their (English) wikipedia article and extract the article's summary and links.

In [3]:
pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=06a936c8c8271fdade75fcc2eef4f1efd9be1259e1fea86df2686f4089aafd81
  Stored in directory: c:\users\etudiant\appdata\local\pip\cache\wheels\c2\46\f4\caa1bee71096d7b0cdca2f2a2af45cacf35c5760bee8f00948
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import wikipedia

wikipedia.search(astronauts[0])

links = []
for i in astronauts:
    print("**This is the Wikipedia summary of", i, ":")
    print("")
    print(wikipedia.summary(i, sentences = 1))
    links += wikipedia.page(i).links[:10]
    print(wikipedia.page(i).links[:10])
    print("")

**This is the Wikipedia summary of Sergey Prokopyev :



C:\Users\Etudiant\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\Etudiant\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "Sergey Prokopyev" may refer to: 
Sergey Prokopyev (beach volleyball)
Sergey Prokopyev (cosmonaut)

#### Question 2.2
Extend your Python program so that it only keeps links that are pointing to wikipedia pages (in any language).

In [5]:
#print(wikipedia.page(links[0]).url)

wlinks = []
errors = []

for l in links:
    try :
        if wikipedia.page(l).url != None:
            wlinks.append(l)
    except wikipedia.exceptions.PageError:
        errors.append(l)

print(errors)
print("")
print(len(wlinks), len(links))
print("")
print(wlinks)
print("")
print(links)

[]

0 0

[]

[]


In [6]:
#Corrections
wiki_links = []

for l in links:
    try:
        page = wikipedia.search(l, results=1)
        wiki_links.append(l)
        print(page)
    except wikipedia.exceptions.PageError as e:
        print(e.options)

#### Question 2.3
Extend your Python program so that it processes these links as follows:
 - it retrieves the corresponding article and then extracts its references


In [7]:
for l in wlinks:
    print("** The references of", l, "are :")
    wikipedia.page(l)
    print(wikipedia.page(l).references) #references : les pages Wiki où l est en lien

#### Question 2.4
Extend your Python program to compute the average number of views for each astronaut's main article.

In [8]:
import wptools

total_views = 0
for x in astronauts:
    page = wptools.page(x)
    nbviews = page.get_more().data['views']
    #print(x, nbviews)
    total_views += int(nbviews)

print("Average number of views:", total_views/len(astronauts))

ModuleNotFoundError: No module named 'wptools'

#### Question 2.5
Export the extracted information in a CSV file having the following fields:

`Astronaut's name ; Article's summary ; links separated by commas ; number of views`

In [9]:
import csv

with open("td4.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter="\t")
    writer.writerow(["Astronaut's name", "Article's summary", "Links", "Number of views"])
    for i in astronauts:
        writer.writerow([i, wikipedia.summary(i, sentences = 1), wikipedia.page(i).links[:10],wptools.page(i).get_more().data['views']])

DisambiguationError: "Sergey Prokopyev" may refer to: 
Sergey Prokopyev (beach volleyball)
Sergey Prokopyev (cosmonaut)

### Exercise 3
#### Question 3.1 
Using Wikipedia, compile the list of UEFA's Intertoto cup winners sorted by country.

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/UEFA_Intertoto_Cup"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the table with the winners information
table = soup.find("table", {"class": "wikitable"})

# Extract the winners and the countries
winners = []
for row in table.find_all("tr")[1:]:
    winner = row.find_all("td")[1].text.strip()
    country = row.find_all("td")[2].text.strip()
    winners.append((winner, country))

# Sort the winners by country
sorted_winners = sorted(winners, key=lambda x: x[1])

# Print the winners
for winner in sorted_winners:
    print(winner[0], "from", winner[1])


#### Question 3.2
Compute the number of Intertoto winners per country.

In [10]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

url = "https://en.wikipedia.org/wiki/UEFA_Intertoto_Cup"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find the table with the winners information
table = soup.find("table", {"class": "wikitable"})

# Extract the winners and the countries
winners = []
for row in table.find_all("tr")[1:]:
    winner = row.find_all("td")[1].text.strip()
    country = row.find_all("td")[2].text.strip()
    winners.append((winner, country))

# Count the number of winners per country
winners_per_country = defaultdict(int)
for winner in winners:
    winners_per_country[winner[1]] += 1

# Print the number of winners per country
for country, count in winners_per_country.items():
    print("{}: {}".format(country, count))


IndexError: list index out of range

### Exercise 4

Let's play with New York Times API! Register here to get your API access token: https://developer.nytimes.com/apis

#### Question 4.1

Extract for each astronaut articles in NYT mentioning them from the year 2022. Create a dictionay where by the astronauts name you could get information on number of articles published about them in 2022, links to the last 10 articles of 2022, and list of keywords which are associated with those articles, including how many times each keyword appeared. 

For example 

    {"John Smith": {"articles": ["nytimes.com/The_Best_Astronaut",
                                 "nytimes.com/Do_we_really_believe_in_space_travel"],
                   "num_articles": 2,
                   "keywords": {"space": 2, "celebrities": 1}}}
                   
<span style="color:red">ATTENTION!</span> Never publish your API access key on github or any other open website! It is only to be used by you. (Well, otherwise the API service could ban you :) )

In [12]:
print(wikipedia.summary("UEFA's Intertoto cup"))
wptools.page("UEFA Intertoto cup")

The UEFA Intertoto Cup (from Latin: Inter, 'between' + German: toto, 'betting pool'), often abbreviated and more known in the German-speaking world as UI Cup and originally called the International Football Cup, was a summer football competition. The competition was discontinued after the 2008 tournament. Teams who originally would have entered the Intertoto Cup now directly enter the qualifying stages of the UEFA Europa League from this point.
The tournament was founded in 1961–62, but was only taken over by UEFA in 1995. Initially, the tournament ended with a single champion, who received the Intertoto Cup.  Starting in 1967, the tournament ended with a number of group winners (7 to 14 winners, see below), who received cash prizes. When UEFA took on the tournament, it became a qualifier for the UEFA Cup, with 2 to 11 Intertoto winners (see below) advancing to the Second qualifying round of the UEFA Cup.
Any club who wished to participate had to apply for entry, with the highest place

NameError: name 'wptools' is not defined

#### Question 4.2. Bonus :) 

What happens when we want to extract links to all the articles and not just the last 10? Extend your program to iterate through pages of the query results :)

In [11]:
import requests
import json

def get_all_articles(query):
    results = []
    has_continue = True
    while has_continue:
        response = requests.get(
            "https://en.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "format": "json",
                "list": "search",
                "srsearch": query,
                "continue": "-||",
                "sroffset": len(results)
            }
        )
        data = json.loads(response.text)
        results.extend(data["query"]["search"])
        has_continue = "continue" in data
    return results

query = "UEFA Intertoto cup winners"
articles = get_all_articles(query)
for article in articles:
    print(article["title"])


List of UEFA Intertoto Cup winners
UEFA Intertoto Cup
2006 UEFA Intertoto Cup
List of UEFA Cup Winners' Cup finals
List of UEFA club competition winners
2008 UEFA Intertoto Cup
List of European Cup and UEFA Champions League finals
List of UEFA Cup and Europa League finals
2007 UEFA Intertoto Cup
2005–06 UEFA Cup
List of UEFA Super Cup matches
2001 UEFA Intertoto Cup
List of UEFA Women's Cup and Women's Champions League finals
UEFA club competition records and statistics
UEFA Europa League
1996 UEFA Cup Winners' Cup final
2002–03 UEFA Cup
List of UEFA club competition winning managers
1999–2000 UEFA Cup
List of UEFA Intertoto Cup winning managers
1999–2000 UEFA Cup
1996 UEFA Cup final
1994 Intertoto Cup
List of football clubs in England by competitive honours won
List of UEFA Europa Conference League finals
Valencia CF
1995–96 UEFA Cup
1997 UEFA Intertoto Cup
A.C. Perugia Calcio
1999–2000 Juventus F.C. season
1999–2000 Juventus F.C. season
2008–09 UEFA Cup
1997 UEFA Intertoto Cup
2004–0